<a href="https://colab.research.google.com/github/ohhongseok/ML-DeepLearning/blob/main/5/5-1/%ED%8A%B8%EB%A6%AC_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>